In [38]:
import yfinance as yf
import ta
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [171]:
class pricecross(Strategy):

    n1 = 2

    def init(self):
        close = self.data.Close
        self.ema = self.I(ta.trend.ema_indicator, pd.Series(close), self.n1)

    def next(self):
        if crossover(df['Close'], self.ema):
            self.buy()
        elif crossover(self.ema, df['Close']):
            self.sell()

In [176]:
df = yf.download('ETH-USD', interval='5m', period='7D')

[*********************100%%**********************]  1 of 1 completed


In [177]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-12-30 00:00:00+00:00,2300.399658,2300.399658,2296.953613,2296.953613,2296.953613,0
2023-12-30 00:05:00+00:00,2296.880859,2299.233887,2296.880859,2299.233887,2299.233887,0
2023-12-30 00:10:00+00:00,2299.263916,2300.287109,2299.186768,2300.287109,2300.287109,0
2023-12-30 00:15:00+00:00,2300.000000,2303.160645,2300.000000,2303.160645,2303.160645,0
2023-12-30 00:20:00+00:00,2302.080811,2303.756104,2302.080811,2303.634277,2303.634277,0


In [178]:
bt = Backtest(df, pricecross, cash=100000, commission=0.00,
exclusive_orders=True)

output = bt.run()

C:\Users\jacob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


In [179]:
output

Start                     2023-12-30 00:00...
End                       2024-01-05 02:20...
Duration                      6 days 02:20:00
Exposure Time [%]                   26.124075
Equity Final [$]                107190.586914
Equity Peak [$]                 107618.314453
Return [%]                           7.190587
Buy & Hold Return [%]               -2.241856
Return (Ann.) [%]                 3636.404051
Volatility (Ann.) [%]             1137.288037
Sharpe Ratio                         3.197435
Sortino Ratio                             inf
Calmar Ratio                      1572.468762
Max. Drawdown [%]                   -2.312545
Avg. Drawdown [%]                    -0.73161
Max. Drawdown Duration        1 days 00:55:00
Avg. Drawdown Duration        0 days 04:45:00
# Trades                                    8
Win Rate [%]                             62.5
Best Trade [%]                       5.358197
Worst Trade [%]                     -0.175231
Avg. Trade [%]                    